In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re
import datetime as dt 

In [2]:
#import outcome file with org_uuid
main_df = pd.read_csv(R"d:/msc-project/data/final/outcome_final.csv",encoding='utf-8',
                         index_col='org_uuid',
                         parse_dates= ['founded_on','seed_date','series_a_date','series_b_date','series_c_date'])
main_processing_df = main_df[['name','outcome']]

In [3]:
#import main organisation csv
org_ori_df = pd.read_csv(R"d:/msc-project/data/pre-processed/organisations_preprocessed.csv",encoding='utf-8',
                         index_col='org_uuid',
                         parse_dates= ['founded_on','went_public_on','acquired_on','closed_on'])

In [4]:
org_ori_df.columns

Index(['name', 'legal_name', 'homepage_url', 'country_code', 'state_code',
       'region', 'city', 'address', 'postal_code', 'status',
       'short_description', 'category_list', 'category_groups_list',
       'num_funding_rounds', 'total_funding_usd', 'total_funding',
       'total_funding_currency_code', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'primary_role', 'num_exits',
       'description', 'ipo_uuid', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code', 'acquisition_uuid', 'acquirer_uuid',
       'acquirer_name', 'acquirer_country_code', 'acquirer_state_code',
       'acquirer_region', 'acquirer_city', 'acquisition_type', 'acquired

In [5]:
org_processing_df = org_ori_df[['facebook_url','linkedin_url','twitter_url']]

In [6]:
#fill NA value with 0 and non NA with 1
org_processing_df[org_processing_df.notnull()] = 1
org_processing_df.fillna(0,inplace=True)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:3215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:4468: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [7]:
main_processing_df = pd.merge(main_processing_df,
                              org_processing_df,
                              on='org_uuid',
                              how='left')

In [8]:
org_processing_df = org_ori_df[['country_code']]

#create column for one-hot-encode for country
column_name = list()
for country_code in list(pd.get_dummies(org_processing_df.country_code).columns):
    column_name.append(str('is_' + country_code.lower()))
    
country_one_hot = pd.get_dummies(org_processing_df.country_code, columns=column_name)
new_column_dict = dict(zip(country_one_hot.columns,column_name))
country_one_hot.rename(columns = new_column_dict,inplace=True)

In [9]:
main_processing_df = pd.merge(main_processing_df,
                              country_one_hot,
                              on='org_uuid',
                              how='left')

In [10]:
main_processing_df

,name,outcome,facebook_url,linkedin_url,twitter_url,is_abw,is_afg,is_ago,is_aia,is_alb,...,is_ukr,is_ury,is_usa,is_uzb,is_ven,is_vnm,is_yem,is_zaf,is_zmb,is_zwe
org_uuid,,,,,,,,,,,,,,,,,,,,,
000014da-0c46-b9cb-0941-3a93c027b119,Resilio,0,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
00002470-bff7-6226-5800-0ca1b3787b6f,Codementor,0,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
000095de-8e2b-82f1-32a7-c222ba3d5682,Ultraprise Loan Technologies,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
0000d497-c93a-eea3-eeb0-a943dfb4f71e,AutoOffer,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0001a8cc-0cdc-4a30-b4d3-da1b425069e1,SnapClarity,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffd7b85-0941-470a-befd-d0acb160c27a,MAD Studios,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fffdb9e1-0ae9-4630-be83-b76e9e6e0ca0,Iamus Technologies,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fffe2508-e9e3-8602-9664-8c6ef2c803de,Social 2 Step,0,1,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
#import event appearance file csv
event_appear_df = pd.read_csv(R"d:/msc-project/data/pre-processed/event_appearances_preprocessed.csv",encoding='utf-8',
                              index_col='event_appear_uuid')
#,parse_dates= ['founded_on','went_public_on','acquired_on','closed_on'])

In [14]:
event_appear_df.head(3)

,event_appear_name,type,event_uuid,event_name,participant_uuid,participant_name,participant_type,appearance_type,short_description
event_appear_uuid,,,,,,,,,
30c4023d-b694-4704-97d7-cd99cd06ecce,Adobe MAX 2014's sponsor - CDW Corporation,event_appearance,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,ae883a31-3739-7eb9-4ce0-463ee070ed79,CDW Corporation,organization,sponsor,NaN
dffe0542-3dac-4228-a997-3eed42cbd67e,Adobe MAX 2014's sponsor - Microsoft,event_appearance,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,fd80725f-53fc-7009-9878-aeecf1e9ffbb,Microsoft,organization,sponsor,NaN
21884e6c-d097-4c54-8513-10536a0ea60d,Adobe MAX 2014's sponsor - Twitter,event_appearance,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,Twitter,organization,sponsor,NaN


In [17]:
top50_events = event_appear_df['event_name'].value_counts()

In [18]:
top50_events

MWC Barcelona                                 3424
Web Summit 2017                               3119
Web Summit 2016                               2076
Mobile World Congress 2016                    1627
RSA Conference 2018                           1408
                                              ... 
RAN World                                        1
Biometric Summit London 2019                     1
IEEE Hot Interconnects Conference 2019           1
ACE Money Transfer Global Remittance Event       1
IPV's 3rd VC Showcase                            1
Name: event_name, Length: 18784, dtype: int64

In [19]:
event_names = event_appear_df.groupby('event_name')

In [23]:
event_names.size()

AttributeError: 'Series' object has no attribute 'sort'